## Loading ROOT libraries

In [1]:
from ROOT import TCanvas, TPad, TFile, TPaveLabel, TPaveText
from ROOT import gROOT
import root_numpy
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (LinearLocator, MultipleLocator, FormatStrFormatter,AutoMinorLocator)
%jsroot on
#ROOT.gStyle.SetOptStat(0)
#ROOT.gStyle.SetOptStat(1111111)

Welcome to JupyROOT 6.18/00


In [2]:
run = [2163]
tip = "3D"

filename = "../reco_files/6040/new_reco_run%05d_%s.root" % (run[0],tip)
tf = ROOT.TFile(filename)
tree = tf.Get('Events')

In [5]:
## Plot parameters
caption = 0
logy = 0
logx = 0
stats = 1
colors = [ROOT.kGray+1]
fac=1
c=0

In [6]:
if (c):
    c.Close()
    ROOT.gSystem.ProcessEvents()
    c = 0

var = "cl_integral"
hist1 = 'Integral'
it=2

ROOT.gStyle.SetOptStat(0)

c = ROOT.TCanvas('','',800,600)

#-------2D Method------#
#it=1
h = ROOT.TH1F('{hist1}'.format(hist1=hist1),'',3200,-800,5400)
#h.Sumw2()
tree.Draw("{vr}*{fac}>>{hist1}".format(fac=fac,vr=var,hist1=hist1),"TMath::Hypot(cl_xmean-1024,(cl_ymean-1024)*1.2)<400 && cl_iteration==2 && cl_nhits>30")
h.SetMarkerStyle(20)
h.SetMarkerSize(0.5)
h.SetMarkerColor(colors[0])
h.SetFillColor(ROOT.kGray)
h.SetLineColor(ROOT.kGray+1)
h.Rebin(10)
h.Rebin(4)
#h.SetFillStyle(3005)

h.Draw("hist")
ROOT.gPad.Update()

h.GetYaxis().SetRangeUser(0,82*2)
h.GetXaxis().SetRangeUser(0,4400)
h.GetXaxis().SetTitle("Cluster Light [ph]")
h.GetYaxis().SetTitle("Number of Clusters")

## FIT Paramters and PLOT
myPolya = ROOT.TF1("myPolya","[0]*ROOT::Math::negative_binomial_pdf(x,[1],[2])",0,10000);
myExp = ROOT.TF1("myExp","[0]*exp(-x/[1])",400,10000);
myPolyaE = ROOT.TF1("myPolyaE","[0]*ROOT::Math::negative_binomial_pdf(x,[1],[2])+[3]*exp(-x/[4])",0,10000);

myPolyaE.SetParameters(10000,0.4,2400,1e2,1e3)
myPolyaE.SetLineColor(ROOT.kGray+2)

h.Fit("myPolyaE","","same",500,4400)

myPolya.SetParameters(myPolyaE.GetParameter(0),myPolyaE.GetParameter(1),myPolyaE.GetParameter(2))
myPolya.SetLineColor(ROOT.kRed)
#myPolya.Draw("same")
#-------Stat Box --------#
dif = 0.28
ypos = 0.35

if stats:
    rMean = ((1-myPolya.GetParameter(1))*myPolya.GetParameter(2))/myPolya.GetParameter(1)
    rSigma = ROOT.TMath.Sqrt((((1-myPolya.GetParameter(1))*myPolya.GetParameter(2))/(myPolya.GetParameter(1)*myPolya.GetParameter(1))))
    
    box = ROOT.TBox(2800,2*69.7,4425,2*80.2);
    box.SetFillStyle(0);
    box.SetLineColor(1);
    box.SetLineWidth(1);
    box.Draw("same");

    t = ROOT.TText();
    t.SetTextFont(82)
    t.SetTextSize(0.04)
    t.SetTextAlign(12);
    t.DrawText(2900,2*77.9,'Mean %.0f' % rMean)
    t.DrawText(2900,2*73.4,"Sigma %.0f" % rSigma)

if caption:
    #-------Legend------#
    difx = 0.245
    xpos = 0.5
    (x1,y1,x2,y2) = (xpos, .75, xpos+difx, .89)
    leg = ROOT.TLegend(x1,y1,x2,y2)
    leg.AddEntry(h,'{hist1}'.format(hist1=hist1),'f')
    leg.Draw()
if logx:
    c.SetLogx()
if logy:
    c.SetLogy()
    
c.SetGrid()
c.Draw()
c.Modified()
c.Update()

#c.Close()

 FCN=117.858 FROM MIGRAD    STATUS=CONVERGED    1294 CALLS        1295 TOTAL
                     EDM=1.44034e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           4.63943e+04   1.93090e+03   9.69414e+00  -1.64399e-07
   2  p1           1.84809e-02   1.55066e-03   5.64126e-07  -2.69227e+00
   3  p2           5.69921e+01   4.83230e+00   1.75714e-03   5.87326e-04
   4  p3           2.63069e+02   1.24578e+02   1.04113e-01  -2.30123e-05
   5  p4           2.99875e+02   5.19800e+01   4.37156e-02  -3.54189e-05


Warning in <TFile::Append>: Replacing existing TH1: Integral (Potential memory leak).
